In [4]:
from aurelio.plotter import plot, plot_single_fold, collect
from plotly.offline import init_notebook_mode
import os
import pandas as pd

In [ ]:
init_notebook_mode(connected=True)

In [19]:
exps = [
    "single_fold_elmo",
    "no_elmo",
    "elmo",
    "single_fold_elmo",
    "single_fold_no_elmo",
    "single_fold_elmo_reduced",
    "single_fold_no_elmo_reduced"
]

records = []
for exp in exps:
    path = "{}/metrics/{}".format(os.getcwd(), exp)
    records += collect(path, exp)
df = pd.DataFrame.from_records(records, index=("name", "model"))
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/eraldo/lia/src/aurelio/metrics/elmo'

In [13]:
df.describe()

,best_epoch,best_validation_em,best_validation_end_acc,best_validation_f1,best_validation_loss,best_validation_span_acc,best_validation_start_acc,epoch,peak_cpu_memory_MB,peak_gpu_0_memory_MB,...,training_loss,training_span_acc,training_start_acc,training_start_epoch,validation_em,validation_end_acc,validation_f1,validation_loss,validation_span_acc,validation_start_acc
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.0,...,75.000000,75.000000,75.000000,75.0,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,9.120000,0.098487,0.148963,0.286201,10.620403,0.045684,0.085004,16.133333,2987.766773,8903.0,...,1.830211,0.621467,0.685847,0.0,0.073230,0.151438,0.270150,12.859906,0.037268,0.076617
std,5.635217,0.070386,0.069609,0.109733,2.053311,0.033190,0.048189,3.496459,410.684721,0.0,...,1.735754,0.333473,0.294330,0.0,0.061125,0.069750,0.100417,1.882025,0.029245,0.045130
min,0.000000,0.011111,0.043956,0.131758,7.838483,0.000000,0.000000,9.000000,2656.580000,8903.0,...,0.046299,0.054348,0.175824,0.0,0.000000,0.043956,0.134373,9.363585,0.000000,0.000000
25%,4.000000,0.039464,0.090421,0.190984,9.004322,0.021978,0.044200,13.000000,2708.152000,8903.0,...,0.307678,0.279108,0.386073,0.0,0.022222,0.088889,0.181563,11.321245,0.011111,0.033333
50%,9.000000,0.066667,0.122222,0.276796,10.092011,0.043956,0.077778,18.000000,2708.152000,8903.0,...,1.194383,0.714286,0.789037,0.0,0.055556,0.142857,0.255710,12.882605,0.033333,0.077778
75%,14.000000,0.144444,0.207570,0.361019,11.642583,0.072944,0.122222,19.000000,3559.654000,8903.0,...,3.447921,0.927280,0.945009,0.0,0.104945,0.198901,0.310200,14.282880,0.063492,0.111111
max,19.000000,0.253968,0.301587,0.510102,15.934134,0.126984,0.188889,19.000000,3632.984000,8903.0,...,5.650990,0.990442,0.996416,0.0,0.222222,0.285714,0.510102,17.235799,0.126984,0.190476


### Curvas de Aprendizado utilizando K-fold com ELMo, Word Embeddings de 600 dimensões e respostas adicionais para treino e dev.

In [ ]:
# metrics_elmo_path = "{}/metrics/elmo".format(os.getcwd())
# plot(metrics_elmo_path)
# # metrics_elmo_path

### Curvas de Aprendizado utilizando K-fold sem ELMo, Word Embeddings de 600 dimensões e respostas adicionais para treino e dev.

In [ ]:
metrics_no_elmo_path = "{}/metrics/no_elmo".format(os.getcwd())
plot(metrics_no_elmo_path)

### Curvas de Aprendizado sem K-Fold e com ELMo, com Word Embeddings de 600 dimensões e respostas adicionais para treino e dev.

In [ ]:
metrics_single_fold_elmo_path = "{}/metrics/single_fold_elmo".format(os.getcwd())
plot(metrics_single_fold_elmo_path)

### Curvas de Aprendizado sem K-Fold e sem ELMo, com Word Embeddings de 600 dimensões e respostas adicionais para treino e dev.

In [ ]:
metrics_single_fold_no_elmo_path = "{}/metrics/single_fold_no_elmo".format(os.getcwd())
plot_single_fold(metrics_single_fold_no_elmo_path)

### Curvas de Aprendizado sem K-Fold e com ELMo, com Word Embeddings de 600 dimensões e respostas adicionais apenas para dev.

In [ ]:
metrics_single_fold_elmo_reduced_path = "{}/metrics/single_fold_elmo_reduced".format(os.getcwd())
plot_single_fold(metrics_single_fold_elmo_reduced_path)

### Curvas de Aprendizado sem K-Fold e sem ELMo, com Word Embeddings de 600 dimensões e respostas adicionais apenas para dev.

In [ ]:
metrics_single_fold_no_elmo_reduced_path = "{}/metrics/single_fold_no_elmo_reduced".format(os.getcwd())
plot_single_fold(metrics_single_fold_no_elmo_reduced_path)